In [1]:
from tkinter import E
import scipy.io
import numpy as np
from matplotlib import pyplot as plt
import neurokit2 as nk2
import neurokit as nk
import csv
import glob
import os
import pandas as pd
from matplotlib.pyplot import axis
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import wfdb


In [2]:

def classify(name,model,features,classes):
    
    #Using 10-KFold cross validation
    kf = KFold(10,shuffle=True)
    score = []
    for trainIndex,testIndex in kf.split(features):
        xTrain, xTest = features[trainIndex],features[testIndex]
        yTrain, yTest = classes[trainIndex],classes[testIndex]
        
        model.fit(xTrain,yTrain)
        prediction = model.predict(xTest)
        score.append( accuracy_score(yTest,prediction))
        
        
    print("The average score for ",name," is ",sum(score)/len(score))
    return model


def preprocessClass(classes,name):
    
    for idx,val in enumerate(classes):
        
        if val == name:
            classes[idx] = '0'
        else:
            classes[idx] = '1'
    
    return classes
    
def classificationProcess(features,classes):
    
    #Calculate the Feature Importance using Forest Tree
    model = ExtraTreesClassifier(100)
    model.fit(features,classes)
    featureImportance = model.feature_importances_
    models = dict()
    specificClass = 'A'
    
    #if specificClass != 'all':
    classes = preprocessClass(classes, specificClass)
    
    #Choose the 5 features with the highest importance
    bestFeatures= []
    for i in range(6):
        max = featureImportance.argmax()
        bestFeatures.append(max)
        featureImportance[max] = 0
     
     #Adjust the features array  to contain only the columns of the best features   
    features = features[:,[bestFeatures[0],bestFeatures[1],bestFeatures[2],bestFeatures[3],bestFeatures[4]]]
    
    ######KNN###########
    parameters = []
    knn = KNeighborsClassifier(4)
    models["KNN"] = classify("KNN",knn, features, classes)
    
    #Using exhaustive GridSearchCV for parameter tuning 
    
    #####SVM############
    parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4], 'C': [1, 10, 100, 1000]}]
    svm = GridSearchCV(SVC(),parameters, cv=5)
    models["SVM"] = classify("SVM",svm,features,classes)
    models["ExtraTreesClassifier"] = classify("SVM",model,features,classes)
    #####Random Forest #####
    parameters = [{'n_estimators': [100, 500, 1000,1500]}]
    forest = GridSearchCV(RandomForestClassifier(),parameters, cv=5)
    models["RF"] = classify("Random Forest",forest,features,classes)
    
    #####Naive Bayes ######
    gauss = GaussianNB()
    models["GAUS"] = classify("Gaussian", gauss, features, classes)
    return models

In [46]:
features = []
classes = []

In [4]:
record_number = '05091'
filepath = 'D:/phd/mit-bih-atrial-fibrillation-database-1.0.0/files/' + record_number
record = wfdb.rdrecord(filepath)
annotation = wfdb.rdann(filepath,'atr', shift_samps=True)

#writeToCsv(features,classes)

#features, classes = loadFromFile()
#models = classificationProcess(features,classes)
points = [79, 1633050, 1633999, 1900120, 1901525, 2270154, 2270587,
    2574098, 2575190, 2591265, 2593950, 2618884, 2619973, 2877400,
    2880801, 4350349, 4360987]

In [5]:
def ProcessSignal(signal, rate):
    peaks = nk2.ecg_findpeaks(signal, rate)
     #Calculate the Heart Rate Variability and extract its measures
    hrv = nk2.hrv(peaks, sampling_rate=rate)
    
    #Calculate the features
    mean = signal.mean() # Mean of the ECG signal
    std = signal.std() # Standard deviation of the ECG signal
    sdNN = list(hrv['HRV_SDNN'])[0] # Standard deviation of RR interval
    meanNN = list(hrv['HRV_MeanNN'])[0] # Mean of RR interval 
    madNN = list(hrv['HRV_MadNN'])[0]  # Median Absolute Deviation (MAD) of the RR intervals.
    pNN50 = list(hrv['HRV_pNN50'])[0] # The number of interval differences of successive RR intervals greater than 50 ms
    RMSSD = list(hrv['HRV_RMSSD'])[0] # the root mean square of the RR intervals
    shannon = list(hrv['HRV_ShanEn'])[0] # Shannon entropy
    sample_entropy = list(hrv['HRV_SampEn'])[0] # Sample Entropy
    if np.isinf(sample_entropy):
        sample_entropy = 0
    return [mean, std, sdNN, meanNN, madNN, pNN50, RMSSD, shannon, sample_entropy]

In [55]:
for i in range(0, int(len(record.p_signal)), 10000):
        signal = record.p_signal[i : i +10000, 0]
        class_signal = 'N'
        for j in range(len(points)):
            if j %2 == 0:
                continue
            if points[j] > i and points[j] < i + 10000:
                print(points[j])
                class_signal = 'A'
        feature = ProcessSignal(signal, 250)
        features.append(feature)
        classes.append(class_signal)

1633050
1900120
2270154
2574098
2591265
2618884
2877400
4350349


In [48]:
len(features), len(classes)

(921, 921)

In [57]:
models = classificationProcess(np.array(features), np.array(classes))

The average score for  KNN  is  0.9905016495817133
The average score for  SVM  is  0.9951138505950278
The average score for  SVM  is  0.9913124484505715
The average score for  Random Forest  is  0.9910458642629905
The average score for  Gaussian  is  0.9611810710498409


In [37]:
np.array(features)[: , [1, 2]]

array([[1.75858172e-01, 2.20371980e+01],
       [1.79547221e-01, 2.22473233e+01],
       [1.76467709e-01, 2.05104746e+01],
       ...,
       [3.95906881e-01, 9.65447896e+01],
       [8.10898198e-01, 2.01067891e+02],
       [2.08851883e+00, 2.57362313e+02]])

In [65]:
models

{'KNN': KNeighborsClassifier(n_neighbors=4),
 'SVM': GridSearchCV(cv=5, estimator=SVC(),
              param_grid=[{'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                           'kernel': ['rbf']}]),
 'ExtraTreesClassifier': ExtraTreesClassifier(),
 'RF': GridSearchCV(cv=5, estimator=RandomForestClassifier(),
              param_grid=[{'n_estimators': [100, 500, 1000, 1500]}]),
 'GAUS': GaussianNB()}

In [64]:
#Calculate the Feature Importance using Forest Tree
model = ExtraTreesClassifier(100)
model.fit(features,classes)
featureImportance = model.feature_importances_
#Choose the 5 features with the highest importance
bestFeatures= []
for i in range(6):
    maxМ = featureImportance.argmax()
    bestFeatures.append(maxМ)
    featureImportance[maxМ] = 0

In [75]:
p = 2270154+1000
signal = record.p_signal[p : p +10000, 0]
r = ProcessSignal(signal, 250)
t = []
t.append(r[bestFeatures[0]])
t.append(r[bestFeatures[1]])
t.append(r[bestFeatures[2]])
t.append(r[bestFeatures[3]])
t.append(r[bestFeatures[4]])
for i in models.items():
    name, model = i
    print(name, model.predict(np.reshape(t, (1, -1))))

KNN ['0']
SVM ['1']
ExtraTreesClassifier ['1']
RF ['1']
GAUS ['1']


In [80]:
from re import M
import keras
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D
from sklearn.metrics import confusion_matrix, classification_report
import joblib

import pickle
import scipy.io
import numpy as np
from os import walk
import os
import csv
import pdb

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

path = 'D:\\ECGAttempts\\Atrial-Fibrilation-Detection\\training2017\\'

batch_size = 10
epochs = 20
num_classes = 10
no_samples = 18286
no_files = 8528
dim_maxima = 18286

In [90]:
def test ():

    test = np.load("/home/iustin/python code/testingset.npy")
    etichete_test = np.load ("/home/iustin/python code/testinglabels.npy")

    model = pickle.load(open("model.pkl", "rb"))
    model.predict(test)

    _, accuracy = model.evaluate(test, etichete_test, batch_size=batch_size, verbose = 1)
    print ("ACCURACY FOR TEST... ")
    print (accuracy)


def label_processing():

    # noua baza de date cu etichetele 1 si O
    d = []

    with open(path + 'REFERENCE.csv') as csvfile:
        content = csv.reader (csvfile, delimiter = ',')
        for c in content:
            if c[1] == 'A':
                c[1] = 1
            else :
                c[1] = 0
            d.append(c)

    with open('db.csv', 'w', newline='') as csvfile:
        s = csv.writer(csvfile, delimiter = ',')
        for r in d:
            s.writerow (r)


def citire_date():

    # citesc datele din fisier
    d = {}

    #csv contine nume imagine si label
    with open('db.csv', 'r', newline='') as csvfile:
        content = csv.reader (csvfile, delimiter = ',')
        for c in content:
            d [c[0]] = c[1]
    
    p1 = int (0.9 * no_files)
    p2 = int (0.7 * p1)

    dtrain = {}
    dtest = {}
    dvalidation = {}

    for cheie, val in d.items():
        if p2 > 0:
            dtrain [path + cheie] = val
            p2 -= 1
            p1 -= 1
        elif p1 > 0:
            dvalidation [path + cheie] = val
            p1 -= 1
        else :
            dtest [path + cheie] = val
    return dtrain, dvalidation, dtest


def preprocesare_train():

    dtrain, _, _ = citire_date() 
    train = []
    etichete_train = np.array ([])
    contor = 0
    print ("Incarc etichetele si datele in memorie ....")
    print ("Start training set")

    #cheia este calea catre imagine, val este labelul
    for cheie, val in dtrain.items():
        mat = scipy.io.loadmat(cheie)
        #incarcarea continutului fisierului din cheie
        #pentru semnalele cu lungimea mai mica de dim maxima, se adauga elemente de 0 pana ce dimensiunea este egala cu dim maxima
        if len(mat['val'][0]) < dim_maxima :
            pad = [float(0)] * (dim_maxima - len(mat['val'][0]))
            train.append([float(x) for x in mat['val'][0]] + pad)
        else :
            train.append([float(x) for x in mat['val'][0]])
        etichete_train = np.append(etichete_train, float(val))
        print ("Done processing Sample {} + Label {} ...".format(contor, contor))
        contor += 1

    print ("Done traning set")
    print ("START - saving train set")
    np.save("trainset", np.array(train))
    print ("END - saving train set")
    print ("START - saving train labels")
    np.save("trainlabels", etichete_train)
    print ("END - saving train labels")


def preprocesare_validation():

    _, dvalidation, _ =  citire_date() 
    validation = []
    etichete_validation = np.array ([])
    contor = 0
    print ("Start validation set")

    for cheie, val in dvalidation.items():
        mat = scipy.io.loadmat(cheie)
        if len(mat['val'][0]) < dim_maxima :
            pad = [0] * (dim_maxima - len(mat['val'][0]))
            validation.append(list(mat['val'][0]) + pad)
        else :
            validation.append(list(mat['val'][0]))
        etichete_validation = np.append(etichete_validation, float(val))
        print ("Done processing Sample {} + Label {} ...".format(contor, contor))
        contor += 1

    print ("Done validation set")
    print ("START - saving validation set")
    np.save("validationset", np.array(validation))
    print ("END - saving validation set")
    print ("START - saving validation labels")
    np.save("validationslabels", np.array(etichete_validation))
    print ("END - saving validation labels")


def preprocesare_test():

    _, _, dtest =  citire_date() 
    test = []
    etichete_test = np.array ([])
    contor = 0
    print ("Start test set")

    for cheie, val in dtest.items():
        mat = scipy.io.loadmat(cheie)
        if len(mat['val'][0]) < dim_maxima :
            pad = [0] * (dim_maxima - len(mat['val'][0]))
            test.append(list(mat['val'][0]) + pad)
        else :
            test.append(list(mat['val'][0]))
        etichete_test = np.append(etichete_test, float(val))
        print ("Done processing Sample {} + Label {} ...".format(contor, contor))
        contor += 1

    print ("Done test set")
    print ("START - saving testing set")
    np.save("testingset", np.array(test))
    print ("END - saving testing set")
    print ("START - saving testing labels")
    np.save("testinglabels", np.array(etichete_test))
    print ("END - saving testing labels")


def cnn ():

    model = Sequential()

    model.add(Conv1D(27, 1, padding='causal', input_shape=(no_samples, 1), activation="relu"))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Dropout(0.25))

    model.add(Conv1D(15, 1, padding='causal', activation="relu"))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Dropout(0.25))

    model.add(Conv1D(4, 1, padding='causal', activation="relu"))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Dropout(0.25))

    model.add(Conv1D(3, 1, padding='causal', activation="relu"))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(30, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(10, activation="softmax"))
    model.add(Dropout(0.25))
    model.add(Dense(2, activation="softmax"))
    
    return model


if __name__ == '__main__':
 
    # preprocesare date
    # preprocesare_train()
    # preprocesare_validation()
    # preprocesare_test()

    print ("Creez si compilez cnn")
    model = cnn()
    model.compile(
        loss = 'sparse_categorical_crossentropy',
        optimizer = 'adam',
        metrics = ['accuracy']
    )
    model.summary()
    #model = pickle.dump(model, open("model.pkl", "wb"))
    #joblib.dump(model, 'modelsalvat.pkl')
    # # test()

Creez si compilez cnn
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_8 (Conv1D)           (None, 18286, 27)         54        
                                                                 
 max_pooling1d_8 (MaxPooling  (None, 9143, 27)         0         
 1D)                                                             
                                                                 
 dropout_12 (Dropout)        (None, 9143, 27)          0         
                                                                 
 conv1d_9 (Conv1D)           (None, 9143, 15)          420       
                                                                 
 max_pooling1d_9 (MaxPooling  (None, 4571, 15)         0         
 1D)                                                             
                                                                 
 dropout_13 (Dropout)        (No

In [82]:
label_processing()

In [84]:
dtrain, dvalidation, dtest = citire_date()

In [88]:
preprocesare_train()
preprocesare_validation()
preprocesare_test()

Incarc etichetele si datele in memorie ....
Start training set
Done processing Sample 0 + Label 0 ...
Done processing Sample 1 + Label 1 ...
Done processing Sample 2 + Label 2 ...
Done processing Sample 3 + Label 3 ...
Done processing Sample 4 + Label 4 ...
Done processing Sample 5 + Label 5 ...
Done processing Sample 6 + Label 6 ...
Done processing Sample 7 + Label 7 ...
Done processing Sample 8 + Label 8 ...
Done processing Sample 9 + Label 9 ...
Done processing Sample 10 + Label 10 ...
Done processing Sample 11 + Label 11 ...
Done processing Sample 12 + Label 12 ...
Done processing Sample 13 + Label 13 ...
Done processing Sample 14 + Label 14 ...
Done processing Sample 15 + Label 15 ...
Done processing Sample 16 + Label 16 ...
Done processing Sample 17 + Label 17 ...
Done processing Sample 18 + Label 18 ...
Done processing Sample 19 + Label 19 ...
Done processing Sample 20 + Label 20 ...
Done processing Sample 21 + Label 21 ...
Done processing Sample 22 + Label 22 ...
Done processin

Done processing Sample 205 + Label 205 ...
Done processing Sample 206 + Label 206 ...
Done processing Sample 207 + Label 207 ...
Done processing Sample 208 + Label 208 ...
Done processing Sample 209 + Label 209 ...
Done processing Sample 210 + Label 210 ...
Done processing Sample 211 + Label 211 ...
Done processing Sample 212 + Label 212 ...
Done processing Sample 213 + Label 213 ...
Done processing Sample 214 + Label 214 ...
Done processing Sample 215 + Label 215 ...
Done processing Sample 216 + Label 216 ...
Done processing Sample 217 + Label 217 ...
Done processing Sample 218 + Label 218 ...
Done processing Sample 219 + Label 219 ...
Done processing Sample 220 + Label 220 ...
Done processing Sample 221 + Label 221 ...
Done processing Sample 222 + Label 222 ...
Done processing Sample 223 + Label 223 ...
Done processing Sample 224 + Label 224 ...
Done processing Sample 225 + Label 225 ...
Done processing Sample 226 + Label 226 ...
Done processing Sample 227 + Label 227 ...
Done proces

Done processing Sample 399 + Label 399 ...
Done processing Sample 400 + Label 400 ...
Done processing Sample 401 + Label 401 ...
Done processing Sample 402 + Label 402 ...
Done processing Sample 403 + Label 403 ...
Done processing Sample 404 + Label 404 ...
Done processing Sample 405 + Label 405 ...
Done processing Sample 406 + Label 406 ...
Done processing Sample 407 + Label 407 ...
Done processing Sample 408 + Label 408 ...
Done processing Sample 409 + Label 409 ...
Done processing Sample 410 + Label 410 ...
Done processing Sample 411 + Label 411 ...
Done processing Sample 412 + Label 412 ...
Done processing Sample 413 + Label 413 ...
Done processing Sample 414 + Label 414 ...
Done processing Sample 415 + Label 415 ...
Done processing Sample 416 + Label 416 ...
Done processing Sample 417 + Label 417 ...
Done processing Sample 418 + Label 418 ...
Done processing Sample 419 + Label 419 ...
Done processing Sample 420 + Label 420 ...
Done processing Sample 421 + Label 421 ...
Done proces

Done processing Sample 605 + Label 605 ...
Done processing Sample 606 + Label 606 ...
Done processing Sample 607 + Label 607 ...
Done processing Sample 608 + Label 608 ...
Done processing Sample 609 + Label 609 ...
Done processing Sample 610 + Label 610 ...
Done processing Sample 611 + Label 611 ...
Done processing Sample 612 + Label 612 ...
Done processing Sample 613 + Label 613 ...
Done processing Sample 614 + Label 614 ...
Done processing Sample 615 + Label 615 ...
Done processing Sample 616 + Label 616 ...
Done processing Sample 617 + Label 617 ...
Done processing Sample 618 + Label 618 ...
Done processing Sample 619 + Label 619 ...
Done processing Sample 620 + Label 620 ...
Done processing Sample 621 + Label 621 ...
Done processing Sample 622 + Label 622 ...
Done processing Sample 623 + Label 623 ...
Done processing Sample 624 + Label 624 ...
Done processing Sample 625 + Label 625 ...
Done processing Sample 626 + Label 626 ...
Done processing Sample 627 + Label 627 ...
Done proces

Done processing Sample 828 + Label 828 ...
Done processing Sample 829 + Label 829 ...
Done processing Sample 830 + Label 830 ...
Done processing Sample 831 + Label 831 ...
Done processing Sample 832 + Label 832 ...
Done processing Sample 833 + Label 833 ...
Done processing Sample 834 + Label 834 ...
Done processing Sample 835 + Label 835 ...
Done processing Sample 836 + Label 836 ...
Done processing Sample 837 + Label 837 ...
Done processing Sample 838 + Label 838 ...
Done processing Sample 839 + Label 839 ...
Done processing Sample 840 + Label 840 ...
Done processing Sample 841 + Label 841 ...
Done processing Sample 842 + Label 842 ...
Done processing Sample 843 + Label 843 ...
Done processing Sample 844 + Label 844 ...
Done processing Sample 845 + Label 845 ...
Done processing Sample 846 + Label 846 ...
Done processing Sample 847 + Label 847 ...
Done processing Sample 848 + Label 848 ...
Done processing Sample 849 + Label 849 ...
Done processing Sample 850 + Label 850 ...
Done proces

...

Done processing Sample 2265 + Label 2265 ...
Done processing Sample 2266 + Label 2266 ...
Done processing Sample 2267 + Label 2267 ...
Done processing Sample 2268 + Label 2268 ...
Done processing Sample 2269 + Label 2269 ...
Done processing Sample 2270 + Label 2270 ...
Done processing Sample 2271 + Label 2271 ...
Done processing Sample 2272 + Label 2272 ...
Done processing Sample 2273 + Label 2273 ...
Done processing Sample 2274 + Label 2274 ...
Done processing Sample 2275 + Label 2275 ...
Done processing Sample 2276 + Label 2276 ...
Done processing Sample 2277 + Label 2277 ...
Done processing Sample 2278 + Label 2278 ...
Done processing Sample 2279 + Label 2279 ...
Done processing Sample 2280 + Label 2280 ...
Done processing Sample 2281 + Label 2281 ...
Done processing Sample 2282 + Label 2282 ...
Done processing Sample 2283 + Label 2283 ...
Done processing Sample 2284 + Label 2284 ...
Done processing Sample 2285 + Label 2285 ...
Done processing Sample 2286 + Label 2286 ...
Done proce

Done processing Sample 165 + Label 165 ...
Done processing Sample 166 + Label 166 ...
Done processing Sample 167 + Label 167 ...
Done processing Sample 168 + Label 168 ...
Done processing Sample 169 + Label 169 ...
Done processing Sample 170 + Label 170 ...
Done processing Sample 171 + Label 171 ...
Done processing Sample 172 + Label 172 ...
Done processing Sample 173 + Label 173 ...
Done processing Sample 174 + Label 174 ...
Done processing Sample 175 + Label 175 ...
Done processing Sample 176 + Label 176 ...
Done processing Sample 177 + Label 177 ...
Done processing Sample 178 + Label 178 ...
Done processing Sample 179 + Label 179 ...
Done processing Sample 180 + Label 180 ...
Done processing Sample 181 + Label 181 ...
Done processing Sample 182 + Label 182 ...
Done processing Sample 183 + Label 183 ...
Done processing Sample 184 + Label 184 ...
Done processing Sample 185 + Label 185 ...
Done processing Sample 186 + Label 186 ...
Done processing Sample 187 + Label 187 ...
Done proces

Done processing Sample 372 + Label 372 ...
Done processing Sample 373 + Label 373 ...
Done processing Sample 374 + Label 374 ...
Done processing Sample 375 + Label 375 ...
Done processing Sample 376 + Label 376 ...
Done processing Sample 377 + Label 377 ...
Done processing Sample 378 + Label 378 ...
Done processing Sample 379 + Label 379 ...
Done processing Sample 380 + Label 380 ...
Done processing Sample 381 + Label 381 ...
Done processing Sample 382 + Label 382 ...
Done processing Sample 383 + Label 383 ...
Done processing Sample 384 + Label 384 ...
Done processing Sample 385 + Label 385 ...
Done processing Sample 386 + Label 386 ...
Done processing Sample 387 + Label 387 ...
Done processing Sample 388 + Label 388 ...
Done processing Sample 389 + Label 389 ...
Done processing Sample 390 + Label 390 ...
Done processing Sample 391 + Label 391 ...
Done processing Sample 392 + Label 392 ...
Done processing Sample 393 + Label 393 ...
Done processing Sample 394 + Label 394 ...
Done proces

Done processing Sample 610 + Label 610 ...
Done processing Sample 611 + Label 611 ...
Done processing Sample 612 + Label 612 ...
Done processing Sample 613 + Label 613 ...
Done processing Sample 614 + Label 614 ...
Done processing Sample 615 + Label 615 ...
Done processing Sample 616 + Label 616 ...
Done processing Sample 617 + Label 617 ...
Done processing Sample 618 + Label 618 ...
Done processing Sample 619 + Label 619 ...
Done processing Sample 620 + Label 620 ...
Done processing Sample 621 + Label 621 ...
Done processing Sample 622 + Label 622 ...
Done processing Sample 623 + Label 623 ...
Done processing Sample 624 + Label 624 ...
Done processing Sample 625 + Label 625 ...
Done processing Sample 626 + Label 626 ...
Done processing Sample 627 + Label 627 ...
Done processing Sample 628 + Label 628 ...
Done processing Sample 629 + Label 629 ...
Done processing Sample 630 + Label 630 ...
Done processing Sample 631 + Label 631 ...
Done processing Sample 632 + Label 632 ...
Done proces

Done processing Sample 803 + Label 803 ...
Done processing Sample 804 + Label 804 ...
Done processing Sample 805 + Label 805 ...
Done processing Sample 806 + Label 806 ...
Done processing Sample 807 + Label 807 ...
Done processing Sample 808 + Label 808 ...
Done processing Sample 809 + Label 809 ...
Done processing Sample 810 + Label 810 ...
Done processing Sample 811 + Label 811 ...
Done processing Sample 812 + Label 812 ...
Done processing Sample 813 + Label 813 ...
Done processing Sample 814 + Label 814 ...
Done processing Sample 815 + Label 815 ...
Done processing Sample 816 + Label 816 ...
Done processing Sample 817 + Label 817 ...
Done processing Sample 818 + Label 818 ...
Done processing Sample 819 + Label 819 ...
Done processing Sample 820 + Label 820 ...
Done processing Sample 821 + Label 821 ...
Done processing Sample 822 + Label 822 ...
Done processing Sample 823 + Label 823 ...
Done processing Sample 824 + Label 824 ...
Done processing Sample 825 + Label 825 ...
Done proces

In [92]:
train = np.load("trainset.npy")
validation = np.load("validationset.npy")

etichete_train = np.load ("trainlabels.npy")
etichete_validation = np.load ("validationslabels.npy")

print ("START TRAIN AND VALIDATION ... ")
model.fit(train, etichete_train, validation_data = (validation, etichete_validation), epochs=epochs, batch_size=batch_size, verbose = 1)
print ("END")

START TRAIN AND VALIDATION ... 
Epoch 1/20
538/538 [==============================] - 83s 149ms/step - loss: 0.4446 - accuracy: 0.9021 - val_loss: 0.3148 - val_accuracy: 0.9231
Epoch 2/20
538/538 [==============================] - 80s 148ms/step - loss: 0.3324 - accuracy: 0.9118 - val_loss: 0.2806 - val_accuracy: 0.9231
Epoch 3/20
538/538 [==============================] - 79s 147ms/step - loss: 0.3162 - accuracy: 0.9118 - val_loss: 0.2740 - val_accuracy: 0.9231
Epoch 4/20
538/538 [==============================] - 79s 147ms/step - loss: 0.3101 - accuracy: 0.9118 - val_loss: 0.2732 - val_accuracy: 0.9231
Epoch 5/20
538/538 [==============================] - 78s 145ms/step - loss: 0.3095 - accuracy: 0.9118 - val_loss: 0.2732 - val_accuracy: 0.9231
Epoch 6/20
538/538 [==============================] - 78s 144ms/step - loss: 0.3056 - accuracy: 0.9118 - val_loss: 0.2729 - val_accuracy: 0.9231
Epoch 7/20
538/538 [==============================] - 79s 148ms/step - loss: 0.3025 - accuracy: 0.

In [91]:
preprocesare_train()

Incarc etichetele si datele in memorie ....
Start training set
Done processing Sample 0 + Label 0 ...
Done processing Sample 1 + Label 1 ...
Done processing Sample 2 + Label 2 ...
Done processing Sample 3 + Label 3 ...
Done processing Sample 4 + Label 4 ...
Done processing Sample 5 + Label 5 ...
Done processing Sample 6 + Label 6 ...
Done processing Sample 7 + Label 7 ...
Done processing Sample 8 + Label 8 ...
Done processing Sample 9 + Label 9 ...
Done processing Sample 10 + Label 10 ...
Done processing Sample 11 + Label 11 ...
Done processing Sample 12 + Label 12 ...
Done processing Sample 13 + Label 13 ...
Done processing Sample 14 + Label 14 ...
Done processing Sample 15 + Label 15 ...
Done processing Sample 16 + Label 16 ...
Done processing Sample 17 + Label 17 ...
Done processing Sample 18 + Label 18 ...
Done processing Sample 19 + Label 19 ...
Done processing Sample 20 + Label 20 ...
Done processing Sample 21 + Label 21 ...
Done processing Sample 22 + Label 22 ...
Done processin

Done processing Sample 207 + Label 207 ...
Done processing Sample 208 + Label 208 ...
Done processing Sample 209 + Label 209 ...
Done processing Sample 210 + Label 210 ...
Done processing Sample 211 + Label 211 ...
Done processing Sample 212 + Label 212 ...
Done processing Sample 213 + Label 213 ...
Done processing Sample 214 + Label 214 ...
Done processing Sample 215 + Label 215 ...
Done processing Sample 216 + Label 216 ...
Done processing Sample 217 + Label 217 ...
Done processing Sample 218 + Label 218 ...
Done processing Sample 219 + Label 219 ...
Done processing Sample 220 + Label 220 ...
Done processing Sample 221 + Label 221 ...
Done processing Sample 222 + Label 222 ...
Done processing Sample 223 + Label 223 ...
Done processing Sample 224 + Label 224 ...
Done processing Sample 225 + Label 225 ...
Done processing Sample 226 + Label 226 ...
Done processing Sample 227 + Label 227 ...
Done processing Sample 228 + Label 228 ...
Done processing Sample 229 + Label 229 ...
Done proces

Done processing Sample 411 + Label 411 ...
Done processing Sample 412 + Label 412 ...
Done processing Sample 413 + Label 413 ...
Done processing Sample 414 + Label 414 ...
Done processing Sample 415 + Label 415 ...
Done processing Sample 416 + Label 416 ...
Done processing Sample 417 + Label 417 ...
Done processing Sample 418 + Label 418 ...
Done processing Sample 419 + Label 419 ...
Done processing Sample 420 + Label 420 ...
Done processing Sample 421 + Label 421 ...
Done processing Sample 422 + Label 422 ...
Done processing Sample 423 + Label 423 ...
Done processing Sample 424 + Label 424 ...
Done processing Sample 425 + Label 425 ...
Done processing Sample 426 + Label 426 ...
Done processing Sample 427 + Label 427 ...
Done processing Sample 428 + Label 428 ...
Done processing Sample 429 + Label 429 ...
Done processing Sample 430 + Label 430 ...
Done processing Sample 431 + Label 431 ...
Done processing Sample 432 + Label 432 ...
Done processing Sample 433 + Label 433 ...
Done proces

Done processing Sample 624 + Label 624 ...
Done processing Sample 625 + Label 625 ...
Done processing Sample 626 + Label 626 ...
Done processing Sample 627 + Label 627 ...
Done processing Sample 628 + Label 628 ...
Done processing Sample 629 + Label 629 ...
Done processing Sample 630 + Label 630 ...
Done processing Sample 631 + Label 631 ...
Done processing Sample 632 + Label 632 ...
Done processing Sample 633 + Label 633 ...
Done processing Sample 634 + Label 634 ...
Done processing Sample 635 + Label 635 ...
Done processing Sample 636 + Label 636 ...
Done processing Sample 637 + Label 637 ...
Done processing Sample 638 + Label 638 ...
Done processing Sample 639 + Label 639 ...
Done processing Sample 640 + Label 640 ...
Done processing Sample 641 + Label 641 ...
Done processing Sample 642 + Label 642 ...
Done processing Sample 643 + Label 643 ...
Done processing Sample 644 + Label 644 ...
Done processing Sample 645 + Label 645 ...
Done processing Sample 646 + Label 646 ...
Done proces

Done processing Sample 821 + Label 821 ...
Done processing Sample 822 + Label 822 ...
Done processing Sample 823 + Label 823 ...
Done processing Sample 824 + Label 824 ...
Done processing Sample 825 + Label 825 ...
Done processing Sample 826 + Label 826 ...
Done processing Sample 827 + Label 827 ...
Done processing Sample 828 + Label 828 ...
Done processing Sample 829 + Label 829 ...
Done processing Sample 830 + Label 830 ...
Done processing Sample 831 + Label 831 ...
Done processing Sample 832 + Label 832 ...
Done processing Sample 833 + Label 833 ...
Done processing Sample 834 + Label 834 ...
Done processing Sample 835 + Label 835 ...
Done processing Sample 836 + Label 836 ...
Done processing Sample 837 + Label 837 ...
Done processing Sample 838 + Label 838 ...
Done processing Sample 839 + Label 839 ...
Done processing Sample 840 + Label 840 ...
Done processing Sample 841 + Label 841 ...
Done processing Sample 842 + Label 842 ...
Done processing Sample 843 + Label 843 ...
Done proces

...

Done processing Sample 4542 + Label 4542 ...
Done processing Sample 4543 + Label 4543 ...
Done processing Sample 4544 + Label 4544 ...
Done processing Sample 4545 + Label 4545 ...
Done processing Sample 4546 + Label 4546 ...
Done processing Sample 4547 + Label 4547 ...
Done processing Sample 4548 + Label 4548 ...
Done processing Sample 4549 + Label 4549 ...
Done processing Sample 4550 + Label 4550 ...
Done processing Sample 4551 + Label 4551 ...
Done processing Sample 4552 + Label 4552 ...
Done processing Sample 4553 + Label 4553 ...
Done processing Sample 4554 + Label 4554 ...
Done processing Sample 4555 + Label 4555 ...
Done processing Sample 4556 + Label 4556 ...
Done processing Sample 4557 + Label 4557 ...
Done processing Sample 4558 + Label 4558 ...
Done processing Sample 4559 + Label 4559 ...
Done processing Sample 4560 + Label 4560 ...
Done processing Sample 4561 + Label 4561 ...
Done processing Sample 4562 + Label 4562 ...
Done processing Sample 4563 + Label 4563 ...
Done proce

Done processing Sample 4742 + Label 4742 ...
Done processing Sample 4743 + Label 4743 ...
Done processing Sample 4744 + Label 4744 ...
Done processing Sample 4745 + Label 4745 ...
Done processing Sample 4746 + Label 4746 ...
Done processing Sample 4747 + Label 4747 ...
Done processing Sample 4748 + Label 4748 ...
Done processing Sample 4749 + Label 4749 ...
Done processing Sample 4750 + Label 4750 ...
Done processing Sample 4751 + Label 4751 ...
Done processing Sample 4752 + Label 4752 ...
Done processing Sample 4753 + Label 4753 ...
Done processing Sample 4754 + Label 4754 ...
Done processing Sample 4755 + Label 4755 ...
Done processing Sample 4756 + Label 4756 ...
Done processing Sample 4757 + Label 4757 ...
Done processing Sample 4758 + Label 4758 ...
Done processing Sample 4759 + Label 4759 ...
Done processing Sample 4760 + Label 4760 ...
Done processing Sample 4761 + Label 4761 ...
Done processing Sample 4762 + Label 4762 ...
Done processing Sample 4763 + Label 4763 ...
Done proce

Done processing Sample 4935 + Label 4935 ...
Done processing Sample 4936 + Label 4936 ...
Done processing Sample 4937 + Label 4937 ...
Done processing Sample 4938 + Label 4938 ...
Done processing Sample 4939 + Label 4939 ...
Done processing Sample 4940 + Label 4940 ...
Done processing Sample 4941 + Label 4941 ...
Done processing Sample 4942 + Label 4942 ...
Done processing Sample 4943 + Label 4943 ...
Done processing Sample 4944 + Label 4944 ...
Done processing Sample 4945 + Label 4945 ...
Done processing Sample 4946 + Label 4946 ...
Done processing Sample 4947 + Label 4947 ...
Done processing Sample 4948 + Label 4948 ...
Done processing Sample 4949 + Label 4949 ...
Done processing Sample 4950 + Label 4950 ...
Done processing Sample 4951 + Label 4951 ...
Done processing Sample 4952 + Label 4952 ...
Done processing Sample 4953 + Label 4953 ...
Done processing Sample 4954 + Label 4954 ...
Done processing Sample 4955 + Label 4955 ...
Done processing Sample 4956 + Label 4956 ...
Done proce

Done processing Sample 5121 + Label 5121 ...
Done processing Sample 5122 + Label 5122 ...
Done processing Sample 5123 + Label 5123 ...
Done processing Sample 5124 + Label 5124 ...
Done processing Sample 5125 + Label 5125 ...
Done processing Sample 5126 + Label 5126 ...
Done processing Sample 5127 + Label 5127 ...
Done processing Sample 5128 + Label 5128 ...
Done processing Sample 5129 + Label 5129 ...
Done processing Sample 5130 + Label 5130 ...
Done processing Sample 5131 + Label 5131 ...
Done processing Sample 5132 + Label 5132 ...
Done processing Sample 5133 + Label 5133 ...
Done processing Sample 5134 + Label 5134 ...
Done processing Sample 5135 + Label 5135 ...
Done processing Sample 5136 + Label 5136 ...
Done processing Sample 5137 + Label 5137 ...
Done processing Sample 5138 + Label 5138 ...
Done processing Sample 5139 + Label 5139 ...
Done processing Sample 5140 + Label 5140 ...
Done processing Sample 5141 + Label 5141 ...
Done processing Sample 5142 + Label 5142 ...
Done proce

Done processing Sample 5319 + Label 5319 ...
Done processing Sample 5320 + Label 5320 ...
Done processing Sample 5321 + Label 5321 ...
Done processing Sample 5322 + Label 5322 ...
Done processing Sample 5323 + Label 5323 ...
Done processing Sample 5324 + Label 5324 ...
Done processing Sample 5325 + Label 5325 ...
Done processing Sample 5326 + Label 5326 ...
Done processing Sample 5327 + Label 5327 ...
Done processing Sample 5328 + Label 5328 ...
Done processing Sample 5329 + Label 5329 ...
Done processing Sample 5330 + Label 5330 ...
Done processing Sample 5331 + Label 5331 ...
Done processing Sample 5332 + Label 5332 ...
Done processing Sample 5333 + Label 5333 ...
Done processing Sample 5334 + Label 5334 ...
Done processing Sample 5335 + Label 5335 ...
Done processing Sample 5336 + Label 5336 ...
Done processing Sample 5337 + Label 5337 ...
Done processing Sample 5338 + Label 5338 ...
Done processing Sample 5339 + Label 5339 ...
Done processing Sample 5340 + Label 5340 ...
Done proce

In [94]:
train.shape

(5372, 18286)

In [95]:
testset = np.load("testingset.npy")

In [96]:
testlabels = np.load("testinglabels.npy")

In [110]:
t = model.predict(testset)

27/27 [==============================] - 1s 39ms/step


In [111]:
mint = 1
for i in t:
    k, d = i
    if k-d < mint:
        mint=k-d
print(mint)

0.81886005


In [104]:
testset.shape

(853, 18286)

In [116]:
testlabels.shape

(853,)

In [117]:
t

array([[0.90943   , 0.09056997],
       [0.90943   , 0.09056997],
       [0.90943   , 0.09056997],
       ...,
       [0.90943   , 0.09056997],
       [0.90943   , 0.09056997],
       [0.90943   , 0.09056997]], dtype=float32)

In [121]:
valTest = model.predict(validation)

72/72 [==============================] - 3s 40ms/step


In [119]:
etichete_validation

array([0., 0., 0., ..., 0., 0., 0.])

In [126]:
vals = 0
for i in range(len(etichete_validation)):
    p, v = valTest[i]
    desc = 1.0
    if p > v : desc = 0.0
    
    if etichete_validation[i] == desc:
        #print(valTest[i], etichete_validation[i])
        vals += 1
vals

2126

In [123]:
valTest.shape

(2303, 2)

In [127]:
vals/2303

0.9231437255753365